In [ ]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
import ollama

In [ ]:
class WebRetriever:
    __HEADERS = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
    }

    __url = None
    __res = None
    __title = None
    __body = None

    def __init__(self, url):
        self.__url = url

    def __request(self):
        self.__res = requests.get(self.__url, headers=self.__HEADERS)

    def __parsing(self):
        soup = BeautifulSoup(self.__res.content, "html.parser")
        self.__title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.__body = soup.body.get_text(separator="\n", strip=True)

    def retrieve(self):
        self.__request()
        self.__parsing()

        return self.__title, self.__body


In [ ]:
class LLMSummarizer:
    __title = None
    __body = None

    def __init__(self):
        pass

    def set_title(self, title):
        self.__title = title

    def set_body(self, body):
        self.__body = body

    def __generate_system_prompt(self):
        system_prompt = "You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown."

        return system_prompt

    def __generate_user_prompt(self):
        user_prompt = f"You are looking at a website titled {self.__title}"
        user_prompt += "\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\n"
        user_prompt += self.__body

        return user_prompt

    def generate_message(self):
        return [
            {"role": "system", "content": self.__generate_system_prompt()},
            {"role": "user", "content": self.__generate_user_prompt()},
        ]

    def summarize(self):
        pass


class ChatGPTSummarizer(LLMSummarizer):
    __response = None
    __openai = None

    def __init__(self):
        super().__init__()
        load_dotenv()
        api_key = os.getenv('OPENAI_API_KEY')
        self.__openai = OpenAI()

    def summarize(self):
        self.__response = self.__openai.chat.completions.create(
            model = "gpt-4o-mini",
            messages = self.generate_message()
        )

        return self.__response.choices[0].message.content


class OllamaSummarizer(LLMSummarizer):
    __MODEL = "llama3.2"
    __response = None

    def __init__(self):
        super().__init__()

    def summarize(self):
        self.__response = ollama.chat(model=self.__MODEL, messages=self.generate_message())

        return self.__response['message']['content']


In [ ]:
class WebSummarizer:
    __url = None
    __web_retriever = None
    __llm_summarizer = None

    def __init__(self, url, llm_summarizer):
        self.__url = url
        self.__web_retriever = WebRetriever(url)
        self.__llm_summarizer = llm_summarizer

    def summarize(self):
        title, body = self.__web_retriever.retrieve()
        self.__llm_summarizer.set_title(title)
        self.__llm_summarizer.set_body(body)

        return self.__llm_summarizer.summarize()

In [61]:
# url = "https://www.prosigmaka.com"
# url = "https://www.prodemy.id"
url = "http://www.detik.com"
# url = "http://www.tempo.co"
# url = "https://www.kompas.com/"

# ChatGPT Summarizer
llm_summarizer = ChatGPTSummarizer()
web_summarizer = WebSummarizer(url, llm_summarizer)
summarized = web_summarizer.summarize()
display(Markdown(summarized))

print("\n" * 5)
print("#" * 180)
print("\n" * 5)

# Ollama Summarizer
llm_summarizer = OllamaSummarizer()
web_summarizer = WebSummarizer(url, llm_summarizer)
summarized = web_summarizer.summarize()
display(Markdown(summarized))

# Summary of Detikcom

Detikcom is an Indonesian news website that provides the latest and most reliable news across various categories, including politics, finance, health, and entertainment. The site features diverse sections such as DetikNews, DetikFinance, and DetikSport, catering to different interests.

## Recent News Highlights:
- **Donald Trump Policies**:
  - Trump announced the U.S. withdrawal from the WHO and initiated a tariff war affecting neighboring economies.
  - He also addressed various policies, including redefining gender recognition and rescinding Biden's Cuba policies.
  - Trump has made controversial statements regarding the relocation of Gaza residents.

- **Other Headlines**:
  - A legislative meeting in Indonesia highlighted issues surrounding nutritious meals for school children.
  - An update on the number of Tuberculosis cases in Indonesia, with significant findings in multiple regions.

## Popular Topics:
Analytics show that discussions around Trump and ongoing social issues are trending, with significant user engagement in comments regarding political actions and decisions.

Detikcom serves as a comprehensive news source, keeping users informed about both national and international events.







####################################################################################################################################################################################








Berikut beberapa berita terpopuler yang terkait dengan topik yang Anda cari:

1. "Perang Tarif Dimulai! Trump Langsung 'Pukul' Ekonomi Tetangga" - Berita ini mengenai pernyataan Presiden Amerika Serikat, Donald Trump, tentang menarik AS keluar dari Organisasi Kesehatan Dunia (WHO) dan mulai perang tarif terhadap negara-negara yang dia anggap tidak setuju dengan kebijaksanaannya.
2. "Trump Hanya Akui 2 Jenis Kelamin di AS, Larang Pergantian Gender" - Berita ini mengenai kembali pernyataan Trump tentang larangan pergantian gender di Amerika Serikat, yang dianggap tidak sesuai dengan hak asasi manusia.
3. "Trump Cabut Kebijakan Era Biden soal Kuba hingga Pemukiman Tepi Barat" - Berita ini mengenai kebijaksanaan Trump tentang membatalkan kebijakan era sebelumnya yang terkait dengan Kuba dan pemukiman tepi barat Amerika Serikat.
4. "Trump Kembalikan Nama Gunung McKinley, Ubah Teluk Meksiko Jadi Teluk Amerika" - Berita ini mengenai perubahan nama gunung dan teluk oleh Trump, yang dianggap tidak sesuai dengan sejarah dan budaya Amerika Serikat.

Namun, perlu diingat bahwa berita-berita tersebut mungkin tidak directly terkait dengan topik "Makan Siang Gratis" yang Anda cari.